In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
# pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_to_pool')
# pre_to_mn_df = connectome_create.load_old_pre_to_mn_df(ext='matched_to_pool')
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
pre_to_mn_df.shape
pre_to_mn_df.sum(axis=0).sum()

In [ ]:
pre_to_mn_df.groupby(level=[0]).size()


In [ ]:

mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]
pre_to_mn_df.head()

new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
pre_to_mn_df = pre_to_mn_df[new_idx]

pre_to_mn_df.head()

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

# Cosine similarity matrices

## All

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_all_grayscale.svg',format='svg')

## Local

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_all_grayscale.svg',format='svg')

## Local Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['local',('L','Leg','femur',All,All,All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['local',('L','Leg','femur',All,All,All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_femur_local_grayscale.svg',format='svg')

## Descending Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['descending',('L','Leg','femur',All,All,All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['descending',('L','Leg','femur',All,All,All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_femur_descending_grayscale.svg',format='svg')

## Intersegmental Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['intersegmental',('L','Leg','femur',All,All,All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['intersegmental',('L','Leg','femur',All,All,All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_femur_intersegmental_grayscale.svg',format='svg')

# Plot number of local synapses vs descending inputs

In [ ]:
fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([0, 2400])
ax.set_xlim([3200, 9000])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
# ax.set_xscale("log");

x = pre_to_mn_df.loc['local',mpool_dict['main_tibia_flexor_muscle']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['main_tibia_flexor_muscle']].sum()
ax.scatter(x,y,color='#2E3191')

x = pre_to_mn_df.loc['local',mpool_dict['tibia_extensor']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['tibia_extensor']].sum()
ax.scatter(x,y,color='#F6921E')

x = pre_to_mn_df.loc['local',mpool_dict['auxiliary_tibia_flexor_E']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['auxiliary_tibia_flexor_E']].sum()
ax.scatter(x,y,color='#2DB515')
fig.savefig('./figpanels/descending_vs_local_inputs.svg',format='svg')

In [ ]:
pre_to_mn_df.loc['local',mpool_dict['tibia_extensor']].sum()

# Bootstrap the cosine similarity metrics
Start with local neurons onto extensors and flexors

In [ ]:
# The starting point

All = slice(None)
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]


mn_mi = femur_df_all.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_all_grayscale.svg',format='svg')

# plot the matrix we're starting with
mn_mi = femur_df_all.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(femur_df_all.to_numpy().transpose())
fig = plt.figure(1, figsize = [10,6])
ax = sns.heatmap(femur_df_all.to_numpy(), xticklabels=mn_mi.segID.to_list(), cmap=utils.white_dense())
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()


#  Bootstrap cosine similarity of MTFs onto each other
The actual similarity for main tibia flexors are very close 1. Shuffling disrupts this

In [ ]:
def get_hat_bar_from_numpy(pmn_conn_mat):
    norm = pmn_conn_mat.copy()
    for c in range(pmn_conn_mat.shape[1]):
        sq = pmn_conn_mat[:,c]**2
        norm[:,c] = np.sqrt(sq.sum())
    hats = pmn_conn_mat/norm

    hat_bar = hats.mean(axis=1)
    hat_bar = hat_bar/np.sqrt((hat_bar**2).sum())
    return hat_bar, hats, pmn_conn_mat 

def residue_norms(hat_bar,mat):
    proj = np.matmul(mat.T,hat_bar)
    residues = mat
    for c in range(residues.shape[1]):
        residues[:,c] = mat-(hat_bar*proj[c])
    residue_norms = np.sqrt((residues**2).sum(axis=0))
    residue_R1_norms = (np.abs(residues)).sum(axis=0)
    return residue_norms,residue_R1_norms

def mix_up_under_mask(mat_in,mask=None,rng=None):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_in[mask] = rng.permutation(mat_in[mask])
    return mat_in.copy()


In [ ]:
femur_df_all

In [ ]:
mft_df = femur_df_all.loc[:,mpool_dict['main_tibia_flexor_muscle']]
mft_df = femur_df_local.loc[:,mpool_dict['main_tibia_flexor_muscle']]
mask_df = pd.DataFrame(data=np.full(mft_df.shape, False),index=mft_df.index,columns=mft_df.columns)

hat_bar, hats, mat = get_hat_bar_from_numpy(mft_df.to_numpy())
mft_hat_proj = np.matmul(hats.T,hat_bar)
actual_proj = np.matmul(hats.T,hat_bar)

# allocate an empty array
N=10000

# This is the full mask, just mix everything up
mask = np.full(mft_df.shape, True)
mixups = np.zeros((N,mft_df.shape[1]))
figname = 'mix_all'

# Just mix up the preferred pool
mask_df.loc[:,:] = False
mask_df.loc['main_tibia_flexor',:] = True
mask = mask_df.to_numpy().copy()
mixups = np.zeros((N,mft_df.shape[1]))
figname = 'mix_pool'

# Just mix up one hemilineage pool
hl = 'A'
mask_df.loc[:,:] = False
mask_df.loc[('main_tibia_flexor',All,hl),:] = True
mask = mask_df.to_numpy().copy()
mixups = np.zeros((N,mft_df.shape[1]))
figname = 'mix_hl_{}'.format(hl)


# Mix up a couple rows
# n_rows = 10
# mask_df.loc[:,:] = False
# mask_df.loc[('main_tibia_flexor',All,All),:].iloc[:n_rows,:] = True
# mask = mask_df.to_numpy().copy()
# mixups = np.zeros((N,mft_df.shape[1]))
# figname = 'mix_rows{}'.format(n_rows)

# Check that the masks are correct
mn_mi = mft_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

if False: # Check the masks
    fig = plt.figure(1, figsize = [6,12])
    ax = sns.heatmap(mask, xticklabels=lbls,)
    cbar = ax.collections[0].colorbar
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Motor Neurons', fontsize =18)
    plt.ylabel('Mixed up vectors')
    plt.show()

rng = np.random.default_rng()

mixupmat = mft_df.to_numpy().copy()
fig = plt.figure(1, figsize = [6,12])
ax = sns.heatmap(np.log10(mixupmat+1), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()
fig.savefig('./figpanels/MFT_local_connectivity.svg',format='svg')

for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
    hat_bar, hats, mat = get_hat_bar_from_numpy(mixup)
    mixups[r,:] = np.matmul(hats.T,hat_bar)


    if r<1: # Check the mixups
        fig = plt.figure(1, figsize = [6,12])
        ax = sns.heatmap(np.log10(mixup+1), xticklabels=lbls,cmap=utils.white_dense())
        cbar = ax.collections[0].colorbar
        cbar.set_label(label = 'log 10 # of synapses',)
        plt.show()
        fig.savefig('./figpanels/MFT_local_conn_random_{}.svg'.format(figname),format='svg')

fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')

# colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
# for m in [1,2,3]:
# exec('mu=mixups_{}'.format(m))
mixupcumsum = mixups.mean(axis=1)
y = np.array(range(len(mixupcumsum))).astype(float)/len(mixupcumsum)
ax.plot(sorted(mixupcumsum),y,color='#000000')
ax.plot(actual_proj.mean()*np.array([1,1]),np.array([0,1]),color='#0000FF')


pct_below_actual = (mixups.mean(axis=1)<actual_proj.mean()).sum() / len(mixups)
p=1-pct_below_actual
ax.text(.94,.5,'p = {:.5f}'.format(p))

fig.savefig(fname='./figpanels/MFT_local_bootstrap_{}.svg'.format(figname),format='svg')


print('{}''%'' of mixups are less than actual projection {}'.format(pct_below_actual*100,actual_proj.sum()))


In [ ]:
def choice_under_mask(mat_in,mask=None,rng=None):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_in[mask] = rng.choice(mat_in[mask],size=mat_in[mask].size)
    return mat_in.copy()

In [ ]:
mask.size

In [ ]:
mft_df = femur_df_local.loc[:,mpool_dict['main_tibia_flexor_muscle']]
mask_df = pd.DataFrame(data=np.full(mft_df.shape, False),index=mft_df.index,columns=mft_df.columns)

hat_bar, hats, mat = get_hat_bar_from_numpy(mft_df.to_numpy())

# allocate an empty array
N=10000

# Just mix up one hemilineage pool
hl = 'A'
classsums = mft_df.groupby('classification_system').sum().sum(axis=1)
class_pct = classsums.sort_values(axis=0,ascending=False)
# class_pct = class_pct/class_pct.sum()
# class_pct.index.values[0:7]

for hl in class_pct.index.values[0]:

    mask_df.loc[:,:] = False
    mask_df.loc[('main_tibia_flexor',All,hl),:] = True
    mask = mask_df.to_numpy().copy()
    mixups = np.zeros((N,mft_df.shape[1]))
    figname = 'mix_hl_{}'.format(hl)

    # Check that the masks are correct
    mn_mi = mft_df.columns.to_frame()
    lbls = utils.mn_labels(mn_mi,depth='rank')

    if True: # Check the masks
        fig = plt.figure(1, figsize = [6,12])
        ax = sns.heatmap(mask, xticklabels=lbls,)
        cbar = ax.collections[0].colorbar
        cbar.set_label(label = 'log 10 # of synapses', size=24)
        plt.xlabel('Motor Neurons', fontsize =18)
        plt.ylabel('Mixed up vectors')
        plt.show()

    rng = np.random.default_rng()

    mixupmat = mft_df.to_numpy().copy()

    for r in range(N): # range(N):
        # Do this for all of the masks
        mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        hat_bar, hats, mat = get_hat_bar_from_numpy(mixup)
        mixups[r,:] = np.matmul(hats.T,hat_bar)


        # if r<1: # Check the mixups
            # fig = plt.figure(1, figsize = [6,12])
            # ax = sns.heatmap(np.log10(mixup+1), xticklabels=lbls,cmap=utils.white_dense())
            # cbar = ax.collections[0].colorbar
            # cbar.set_label(label = 'log 10 # of synapses',)
            # plt.show()
            # fig.savefig('./figpanels/MFT_local_conn_random_{}.svg'.format(figname),format='svg')

    fig, ax = plt.subplots( 1, 1, figsize=(8,8))
    # ax = plt.subplot2grid((1,1),(0,0))
    ax.set_ylim([-.1, 1.1])
    # ax.set_xlim([.945, .96])
    ax.set_ylabel('{}: Cummulative fraction'.format(hl))
    ax.set_xlabel('Sum of cosine similarity for MTF')

    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    mixupcumsum = mixups.mean(axis=1)
    y = np.array(range(len(mixupcumsum))).astype(float)/len(mixupcumsum)
    ax.plot(sorted(mixupcumsum),y,color='#000000')
    ax.plot(actual_proj.mean()*np.array([1,1]),np.array([0,1]),color='#0000FF')


    pct_below_actual = (mixups.mean(axis=1)<actual_proj.mean()).sum() / len(mixups)
    p=1-pct_below_actual
    ax.text(.94,.5,'p = {:.5f}'.format(p))

    fig.savefig(fname='./figpanels/MFT_local_bootstrap_{}.svg'.format(figname),format='svg')


    print('{}''%'' of mixups are less than actual projection {}'.format(pct_below_actual*100,actual_proj.sum()))


In [ ]:
hl

In [ ]:
mft_df.index.get_level_values('classification_system').unique()

# Find a couple of interesting premotor neurons that innervate the flexor pool
Plot their inputs to the flexor pool    

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor']]
mft_df = mft_df.iloc[0:50,:]

mft_df_mtf = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df_mtf = mft_df_mtf.iloc[0:50,:]

ratiodf = mft_df_mtf.divide(mft_df_mtf.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

In [ ]:
mft_df.loc[(All,All,'E',All)]
mft_df.sum(axis=1)
# mft_df[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]

In [ ]:
cool_local_pMN1 = mft_df.loc[('main_tibia_flexor','Glu','B','21A',All)]
cool_local_pMN1_id = cool_local_pMN1.index.get_level_values('segID')[0]
cool_local_pMN1_id

In [ ]:
cool_local_pMN2 = mft_df.loc[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]
cool_local_pMN2_id = cool_local_pMN2.index.get_level_values('segID')[2]
cool_local_pMN2_id

In [ ]:
cool_local_pMN2

In [ ]:
cool_local_pMN3 = mft_df.loc[('main_tibia_flexor','Ach','C','3A',All)]
cool_local_pMN3
cool_local_pMN3_id = cool_local_pMN3.index.get_level_values('segID')[2]
cool_local_pMN3_id

In [ ]:
cool_local_pMN4 = mft_df.loc[('main_tibia_flexor','Glu','R','R1',All)]
cool_local_pMN4_id = cool_local_pMN4.index.get_level_values('segID')[0]
cool_local_pMN4

In [ ]:
def build_synapses_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url


def build_spheres_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='pre_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
timestamp = connectome_create.get_timestamp()

flexor_rnk = mft_df.columns.get_level_values('rank').to_list()
flexor_rnk[1] = '1'
flexor_id = mft_df.columns.get_level_values('segID').to_list()

for cool_pmn in [cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]:
    render_neurons = [cool_pmn]

    syn_df_dict = {}
    clr_dict = {}
    clr_list = ['#AAAAFF','#6666FF','#4444CC','#2222AA','#0000AA']
    for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
        syn_df = client.materialize.synapse_query(post_ids = id, pre_ids=cool_pmn, timestamp=timestamp)
        syn_df['radius'] = 180
        syn_df_dict[rnk] = syn_df
        clr_dict[rnk] = clr

    # print(build_synapses_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))
    print(build_spheres_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))

# Compare ratio of inputs

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]
mft_df = mft_df.iloc[0:50,:]

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(mft_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(mft_df.to_numpy(), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()
plt.savefig('./figpanels/tibiaflexA_heat_map.svg',format='svg')

import ptitprince as pt

ratiodf = mft_df.divide(mft_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

x = [i for i in range(mft_df.shape[1])]

fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.plot(x,ratiodf.to_numpy().T,color='#CCCCCC',marker='.')

ax.plot(x,np.median(ratiodf.to_numpy(),axis=0),color='#000000',marker='.',ls='')

# total = mft_df.sum(axis=0)
# total = total/total.sum()
# ax.plot(x,total.to_numpy(),color='#FF0000',marker='.',ls='')

ax.set_xlabel('Rank of tibia flexor MN')
ax.set_ylabel('Ratio of input, compare to Fast flexor')

ratiodf.columns = ratiodf.columns.droplevel(['side','nerve','segment','function','muscle','segID'])
ranks = ratiodf.copy()
for c in range(ranks.shape[1]):
    ranks.iloc[:,c] = c

df = pd.concat([pd.concat([ranks.iloc[:,x] for x in range(ranks.shape[1])]),pd.concat([ratiodf.iloc[:,x] for x in range(ratiodf.shape[1])])],axis=1)
df.columns=['rank','ratio']
df['total'] = mft_df.sum(axis=1)
# df['total'] = np.log10(mft_df.sum(axis=1)+1)

dx = "rank"; dy = "ratio"; ort = "v"; pal = "Set2"; sigma = .2
f, ax = plt.subplots(figsize=(32, 7))

ax=pt.RainCloud(x = dx, y = dy, data = df, palette = pal, bw = sigma,
                 width_viol = .6, ax = ax, orient = ort, move = .2,pointplot=True)

plt.title("Relative of synapse counts compared to Fast MN")

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/tibflexA_prop_conn_raincloud.svg')

# fig, ax = plt.subplots( 1, 1, figsize=(20,8))
# sns.swarmplot(data=df, x="rank", y="ratio", hue="total",palette="binary",ax=ax)
# sns.stripplot(data=df, x="rank", y="ratio", hue="total",palette="binary",ax=ax)# ax.get_legend().remove()
# ax.set_ylim([-.1,.75])
# plt.savefig('./figpanels/tibflexA_prop_conn_stripplot.svg')


In [ ]:

fig, ax = plt.subplots( 1, 1, figsize=(20,8))
sns.scatterplot(data=df.iloc[::-1], x="rank", y="ratio", hue="total",palette=utils.white_dense(),ax=ax)
# ax = sns.scatterplot(data=tips, x="sex", y="total_bill", hue="total_bill", palette="viridis", legend=False)
pts = ax.collections[0]
pts.set_offsets(pts.get_offsets() + np.c_[np.random.uniform(-.2, .2, len(df)), np.zeros(len(df))])
ax.margins(x=.5)
ax.autoscale_view()
ax.get_legend().remove()
norm = plt.Normalize(0,df['total'].max())
sm = plt.cm.ScalarMappable(cmap=utils.white_dense(), norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)

for cool_pmn in [cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]:
    pmndf = df.loc[(All,All,All,All,cool_pmn)].copy().reset_index()
    sns.lineplot(data=pmndf,x="rank",y='ratio')

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/tibiaflexA_prop_conn_stripplot.svg')


In [ ]:
# df.loc[(All,All,All,All,cool_pmn)]
pmndf.reset_index()

In [ ]:
[cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]

In [ ]:
import scipy

scipy.stats.mannwhitneyu(ratiodf.iloc[:,7],ratiodf.iloc[:,8],alternative='two-sided',)

In [ ]:
scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4],ratiodf.iloc[:,5],ratiodf.iloc[:,6],ratiodf.iloc[:,7],ratiodf.iloc[:,8])

In [ ]:
import scikit_posthocs 

scikit_posthocs.posthoc_conover(df,val_col= 'ratio',group_col='rank',p_adjust='holm')


## Just the main tibia flexors

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df = mft_df.iloc[0:50,:]

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(mft_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(mft_df.to_numpy(), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = '# of synapses',)
plt.savefig('./figpanels/mtf_heat_map.svg',format='svg')
plt.show()


import ptitprince as pt

ratiodf = mft_df.divide(mft_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

x = [i for i in range(mft_df.shape[1])]

fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.plot(x,ratiodf.to_numpy().T,color='#CCCCCC',marker='.')

ax.plot(x,np.median(ratiodf.to_numpy(),axis=0),color='#000000',marker='.',ls='')

# total = mft_df.sum(axis=0)
# total = total/total.sum()
# ax.plot(x,total.to_numpy(),color='#FF0000',marker='.',ls='')

ax.set_xlabel('Rank of tibia flexor MN')
ax.set_ylabel('Ratio of input, compare to Fast flexor')

ratiodf.columns = ratiodf.columns.droplevel(['side','nerve','segment','function','muscle','segID'])
ranks = ratiodf.copy()
for c in range(ranks.shape[1]):
    ranks.iloc[:,c] = c

df = pd.concat([pd.concat([ranks.iloc[:,x] for x in range(ranks.shape[1])]),pd.concat([ratiodf.iloc[:,x] for x in range(ratiodf.shape[1])])],axis=1)
df.columns=['rank','ratio']
df['total'] = mft_df.sum(axis=1)
df['total'] = np.log10(mft_df.sum(axis=1)+1)

dx = "rank"; dy = "ratio"; ort = "v"; pal = "Set2"; sigma = .2
f, ax = plt.subplots(figsize=(32, 7))

ax=pt.RainCloud(x = dx, y = dy, data = df, palette = pal, bw = sigma,
                 width_viol = .6, ax = ax, orient = ort, move = .2)
ax.set_ylim([-.1,.75])
plt.title("Ratio of synapse counts compared to Fast MN")
plt.savefig('tibflexA_heat_map.svg')
plt.savefig('./figpanels/mtf_prop_conn_raincloud.svg')

fig, ax = plt.subplots( 1, 1, figsize=(20,8))
sns.scatterplot(data=df.iloc[::-1], x="rank", y="ratio", hue="total",palette=utils.white_dense(),ax=ax)
# ax = sns.scatterplot(data=tips, x="sex", y="total_bill", hue="total_bill", palette="viridis", legend=False)
pts = ax.collections[0]
pts.set_offsets(pts.get_offsets() + np.c_[np.random.uniform(-.2, .2, len(df)), np.zeros(len(df))])
ax.margins(x=.5)
ax.autoscale_view()
ax.get_legend().remove()
norm = plt.Normalize(0,df['total'].max())
sm = plt.cm.ScalarMappable(cmap=utils.white_dense(), norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/mtf_prop_conn_stripplot.svg')


In [ ]:
len(df)

In [ ]:
import scipy

scipy.stats.mannwhitneyu(ratiodf.iloc[:,0],ratiodf.iloc[:,4],alternative='less',)

In [ ]:
scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4])

In [ ]:
scikit_posthocs.posthoc_conover(df,val_col= 'ratio',group_col='rank',p_adjust='holm')


In [ ]:
df[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]

# Low projection onto B and E

In [ ]:
def get_hat_bar_from_numpy(pmn_conn_mat):
    norm = pmn_conn_mat.copy()
    for c in range(pmn_conn_mat.shape[1]):
        sq = pmn_conn_mat[:,c]**2
        norm[:,c] = np.sqrt(sq.sum())
    hats = pmn_conn_mat/norm

    hat_bar = hats.mean(axis=1)
    hat_bar = hat_bar/np.sqrt((hat_bar**2).sum())
    return hat_bar, hats, pmn_conn_mat 

def residue_norms(hat_bar,mat):
    proj = np.matmul(mat.T,hat_bar)
    residues = mat
    for c in range(residues.shape[1]):
        residues[:,c] = mat-(hat_bar*proj[c])
    residue_norms = np.sqrt((residues**2).sum(axis=0))
    residue_R1_norms = (np.abs(residues)).sum(axis=0)
    return residue_norms,residue_R1_norms

def mix_up_under_mask(mat_in,mask=None,rng=None):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_in[mask] = rng.permutation(mat_in[mask])
    return mat_in.copy()

In [ ]:
mft_df = femur_df_all.loc[:,mpool_dict['main_tibia_flexor_muscle']]
mft_B_df = femur_df_local.loc[:,mpool_dict['auxiliary_tibia_flexor_B']]
mft_E_df = femur_df_local.loc[:,mpool_dict['auxiliary_tibia_flexor_E']]

mft_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mft_df.to_numpy())
auxB_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mft_B_df.to_numpy())
auxE_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mft_E_df.to_numpy())

actual_pairwise = np.array([np.matmul(mft_hat_bar.T,auxB_hat_bar),np.matmul(auxB_hat_bar.T,auxE_hat_bar),np.matmul(mft_hat_bar.T,auxE_hat_bar)])

flexor_df = femur_df_all.loc[:,mpool_dict['flexors_mft_B_E']]
mask_df = pd.DataFrame(data=np.full(flexor_df.shape, False),index=flexor_df.index,columns=flexor_df.columns)
# allocate an empty array
N=10000

# This is the full mask, just mix everything up
mask_df.loc[:,:] = False
mask_df.loc[:,:] = True
mask = mask_df.to_numpy().copy()
mixups = np.zeros((N,len(actual_pairwise)))
figname = 'mix_all'

# # Just smear out up the preferred pool
# mask_df.loc[:,:] = False
# mask_df.loc['main_tibia_flexor',:] = True
# mask = mask_df.to_numpy().copy()
# mixups = np.zeros((N,len(actual_pairwise)))
# figname = 'mix_pool'

# # # Just mix up one hemilineage pool
# hl = 'A'
# mask_df.loc[:,:] = False
# mask_df.loc[('main_tibia_flexor',hl),:] = True
# mask = mask_df.to_numpy().copy()
# mixups = np.zeros((N,len(actual_pairwise)))
# figname = 'mix_hl_{}'.format(hl)

# Mix up a couple rows
# n_rows = 10
# mask_df.loc[:,:] = False
# mask_df.loc[('main_tibia_flexor',All,All),:].iloc[:n_rows,:] = True
# mask = mask_df.to_numpy().copy()
# mixups = np.zeros((N,len(actual_pairwise)))
# figname = 'mix_rows{}'.format(n_rows)

# Check that the masks are correct
mn_mi = flexor_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

if True: # Check the masks
    fig = plt.figure(1, figsize = [6,12])
    ax = sns.heatmap(mask, xticklabels=lbls,)
    cbar = ax.collections[0].colorbar
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Motor Neurons', fontsize =18)
    plt.ylabel('Mixed up vectors')
    plt.show()

rng = np.random.default_rng()

mixupmat = flexor_df.to_numpy().copy()
fig = plt.figure(1, figsize = [6,12])
ax = sns.heatmap(np.log10(mixupmat+1), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()
fig.savefig('./figpanels/flexorMBE_local_connectivity.svg',format='svg')


for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)

    mft_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mixup[:,0:4])
    auxB_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mixup[:,4:9])
    auxE_hat_bar,tmp1,tmp2 = get_hat_bar_from_numpy(mixup[:,9:])

    mixups[r,:] = [np.matmul(mft_hat_bar.T,auxB_hat_bar),np.matmul(auxB_hat_bar.T,auxE_hat_bar),np.matmul(mft_hat_bar.T,auxE_hat_bar)]

    if r<1: # Check the mixups
        fig = plt.figure(2, figsize = [6,12])
        ax = sns.heatmap(np.log10(mixup+1), xticklabels=lbls,cmap=utils.white_dense())
        cbar = ax.collections[0].colorbar
        cbar.set_label(label = 'log 10 # of synapses',)
        plt.show()
        fig.savefig('./figpanels/flexorMBE_local_conn_random_{}.svg'.format(figname),format='svg')

fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([0, .6])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')

# colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
# for m in [1,2,3]:
# exec('mu=mixups_{}'.format(m))
mixupcumsum = mixups.mean(axis=1)
y = np.array(range(len(mixupcumsum))).astype(float)/len(mixupcumsum)
ax.plot(sorted(mixupcumsum),y,color='#000000')
ax.plot(actual_pairwise.mean()*np.array([1,1]),np.array([0,1]),color='#0000FF')


pct_below_actual = (mixups.mean(axis=1)<actual_pairwise.mean()).sum() / len(mixups)
p=1-pct_below_actual
ax.text(.5,.5,'p = {:.5f}'.format(p))

fig.savefig(fname='./figpanels/flexorMBE_local_bootstrap_{}.svg'.format(figname),format='svg')


print('{}''%'' of mixups are less than actual projection {}'.format(pct_below_actual*100,actual_pairwise.mean()))
